In [59]:
from azureml.core import Workspace, Dataset, Datastore
import pandas as pd
import logging


def get_data():
    subscription_id = 'd2706c67-acfc-4bd3-9067-3ff6ac190bc9'
    resource_group = 'capstone-project'
    workspace_name = 'capstone-project'

    workspace = Workspace(subscription_id, resource_group, workspace_name)

    datastore = Datastore.get(workspace, "workspaceworkingdirectory")
    dataset = Dataset.Tabular.from_delimited_files(path=(datastore, 'Users/hualcosa/nd00333-capstone/data/Walmart Data Analysis and Forcasting.csv'))
    df = dataset.to_pandas_dataframe() 
    return df

In [63]:
def process_data(df):
    '''
    This function formats the dataframe, adding past 8 weeks of sales as lagged features
    and 4 weeks of future sales as the label column.

    Return:
    X. Training data with features + lagged sales
    y. vector with the next 4 weeks of sales
    '''
    
    df_with_windows = []
    for store_num in df.Store.unique():
        store_df = df[df.Store == store_num].copy()
        # making lag features
        for i in range(1, 9):
            store_df[f'Weekly_Sales_t-{i}'] = store_df['Weekly_Sales'].shift(i)
        # making future_time_steps
        for i in range(1,4):
            store_df[f'Weekly_Sales_t+{i}'] = store_df['Weekly_Sales'].shift(-i)

        df_with_windows.append(store_df)

    df_with_windows = pd.concat(df_with_windows).dropna()
    # renaming first future value, to follow the same pattern as the other columns
    df_with_windows.rename(columns={"Weekly_Sales":"Weekly_Sales_t+0"}, inplace=True)
    df_with_windows = df_with_windows[['Store', 'Date', 'Holiday_Flag', 'Temperature',
                                        'Fuel_Price', 'CPI', 'Unemployment', 'Weekly_Sales_t-1',
                                        'Weekly_Sales_t-2', 'Weekly_Sales_t-3', 'Weekly_Sales_t-4',
                                        'Weekly_Sales_t-5', 'Weekly_Sales_t-6', 'Weekly_Sales_t-7',
                                        'Weekly_Sales_t-8', 'Weekly_Sales_t+0','Weekly_Sales_t+1', 'Weekly_Sales_t+2',
                                        'Weekly_Sales_t+3']]

    # future columns filter
    ftr = df_with_windows.columns.str.match(r'.+t\+\d')
    # making label vector
    y = df_with_windows.loc[:, ftr].apply(lambda row: list(row), axis=1)
    df_with_windows = df_with_windows.loc[:, ~ftr]
    # making training data
    X = df_with_windows.values

    return X, y


In [50]:
y

8       [1594968.28, 1545418.53, 1466058.28, 1391256.12]
9       [1545418.53, 1466058.28, 1391256.12, 1425100.71]
10      [1466058.28, 1391256.12, 1425100.71, 1603955.12]
11       [1391256.12, 1425100.71, 1603955.12, 1494251.5]
12       [1425100.71, 1603955.12, 1494251.5, 1399662.07]
                              ...                       
6427         [766512.66, 702238.27, 723086.2, 713173.95]
6428         [702238.27, 723086.2, 713173.95, 733455.07]
6429         [723086.2, 713173.95, 733455.07, 734464.36]
6430        [713173.95, 733455.07, 734464.36, 718125.53]
6431        [733455.07, 734464.36, 718125.53, 760281.43]
Length: 5940, dtype: object

In [64]:
def main():
    logging.basicConfig(
        level=logging.INFO,
        format="%(name)s - %(asctime)s - %(levelname)s - %(message)s",
        datefmt="%d-%b-%y %H:%M:%S",
    )

    logging.info('Getting data...')
    df = get_data()
    logging.info('formatting data...')
    X, y = process_data(df)
    print(f"X shape: {X.shape}")
    print(f"y shape: {y.shape}")
    # logging.info('training model...')

In [65]:
if __name__ == '__main__':
    main()

X shape: (5940, 15)
y shape: (5940,)


In [58]:
main()

In [56]:
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Weekly_Sales_t-1,Weekly_Sales_t-2,Weekly_Sales_t-3,Weekly_Sales_t-4,Weekly_Sales_t-5,Weekly_Sales_t-6,Weekly_Sales_t-7,Weekly_Sales_t-8,Weekly_Sales_t+1,Weekly_Sales_t+2,Weekly_Sales_t+3
0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1641957.44,1611968.17,1409727.59
1,1,2010-02-12,1641957.44,1,38.51,2.548,211.242170,8.106,1643690.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1611968.17,1409727.59,1554806.68
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,1641957.44,1643690.90,NaN,NaN,NaN,NaN,NaN,NaN,1409727.59,1554806.68,1439541.59
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106,1611968.17,1641957.44,1643690.90,NaN,NaN,NaN,NaN,NaN,1554806.68,1439541.59,1472515.79
4,1,2010-03-05,1554806.68,0,46.50,2.625,211.350143,8.106,1409727.59,1611968.17,1641957.44,1643690.9,NaN,NaN,NaN,NaN,1439541.59,1472515.79,1404429.92
